In [56]:
import requests
import random
import urllib
import json
import sys
import getpass

username = input("JAMA username>")
password = getpass.getpass("JAMA password>")


JAMA username>chris.winsor
JAMA password>········


In [53]:
base_url = "https://rapidmicrobiosystems.jamacloud.com/rest/latest/"


In [54]:


def get(url):
    response = requests.get(url, auth=(username, password))
    return json.loads(response.text)

# arrgh - the JAMA api only gets up to 50 items - need to do multiple requests
def get_all_data(url):
    # from https://community.jamasoftware.com/blogs/john-lastname/2017/09/29/managing-multiple-pages-of-results-in-the-jama-rest-api

    all_data = []
    allowed_results = 10
    max_results = "maxResults=" + str(allowed_results)
    
    result_count = -1
    start_index = 0
    
    while result_count != 0:
        start_at = "startAt=" + str(start_index)
        
        part_url = url + "?" + start_at + "&" + max_results 
        response = get(part_url)
        
        page_info = response["meta"]["pageInfo"]
        start_index = page_info["startIndex"] + allowed_results
        result_count = page_info["resultCount"]

        all_data.extend(response["data"])
    return all_data

    
    

def put(url, payload):
    response = requests.put(url, json=payload, auth=(username, password))
    return response.status_code

def get_id(string_to_find):
    url = base_url + "abstractitems?contains=" + string_to_find
    json_response = get(url)

    total_results = json_response["meta"]["pageInfo"]["totalResults"]

    if total_results == 1:
        data = json_response["data"]
        item = data[0]
        return item["id"]

    else:
        print("string_to_find wasn't unique")
        sys.exit(1)

def update_item(item_id):
    set_lock_state(True, item_id)
    item = get_item(item_id)
    item["fields"]["description"] += test_results()
    put_item(item_id, item)
    set_lock_state(False, item_id)

def set_lock_state(new_lock_state, item_id):
    payload = {
        "locked": new_lock_state
    }
    url = base_url + "items/" + str(item_id) + "/lock"
    put(url, payload)

def get_item(item_id):
    url = base_url + "items/" + str(item_id)
    json_response = get(url)

    return json_response["data"]

def put_item(item_id, item):
    print("put_item ", item_id)
    url = base_url + "items/" + str(item_id)
    status_code = put(url, item)
    if status_code < 400:
        print("Success", str(status_code))
    else:
        print("Fail", str(status_code))

def test_results():
    template = "<p><strong>Imported test results:</strong></p>Status:&nbsp;"
    test_passed = random.randrange(0,2)
    if test_passed == 1:
        return template + "pass"
    return template + "fail"


In [69]:
# get a list of the tests we need to update

verifications = get(base_url + "items/9384")
print(verifications["data"]["fields"]["name"])

verif_children = get_all_data(base_url + "items/9384/children")

for vidx,vc in enumerate(verif_children):

    
    srd_children = get_all_data(base_url + "items/" +  str(vc["id"]) + "/children")
    for sidx,sc in enumerate(srd_children):
        #print("~~~~~~", sc["id"], " ", sc["fields"]["name"])

        if sc["fields"]["name"] == "Overview":
            print()
            print(str(vidx), " ", vc["id"], " ", vc["fields"]["name"])
            print(sc["id"], " ", sc["fields"]["name"])


Verifications

0   10583   SRD1059  Help about dialog box
10584   Overview

1   10604   SRD1002_CW
10605   Overview

2   10654   SRD1029 To have better message when running empty Audit report
10655   Overview

3   10853   SRD1032_CW Name Fields - In Name fields, please make validation messages more specific so the user knows why
10854   Overview

4   10946   SRD1036_CW
10947   Overview

5   10961   SRD1039_CW
10962   Overview

6   10965   SRD1037  IT Settings, unable to save new local destination for logs after changing location and selecting Copy Now button.
10966   Overview

7   10969   SRD1041 Copy and Email Logs should accept and send out to multiple recipients
10970   Overview

8   10981   SRD1043 Email logs: Only last 3 numbers of System serial number is displayed in the email's subject and body . It should display full system serial number - E12345678
10982   Overview

9   10987   SRD1044_CW
10988   Overview

10   10992   SRD1046 Copy and Email Diagnostic Files has inconsistent 

In [68]:
item = get_item(10583)
print(item)


{'id': 10583, 'documentKey': 'GDP-FLD-57', 'globalId': 'GID-48981', 'itemType': 32, 'project': 50, 'createdDate': '2018-02-22T22:18:19.000+0000', 'modifiedDate': '2018-04-22T02:26:51.000+0000', 'lastActivityDate': '2018-04-22T02:26:51.000+0000', 'createdBy': 16, 'modifiedBy': 14, 'fields': {'documentKey': 'GDP-FLD-57', 'globalId': 'GID-48981', 'name': 'SRD1059  Help about dialog box', 'description': ''}, 'resources': {'self': {'allowed': ['GET', 'PUT', 'PATCH', 'DELETE']}}, 'location': {'sortOrder': 0, 'globalSortOrder': 24552, 'sequence': '1.8.4.1', 'parent': {'item': 9384}}, 'lock': {'locked': False, 'lastLockedDate': '2018-04-22T02:26:52.000+0000'}, 'childItemType': 89, 'type': 'items'}
